# Create dataset
* Takes post df and comments df from download_reddit_posts_and_comments.ipynb and finds first suicidal post, counts distribution of subsequent posts, and adds comments
* Feature extraction


In [ ]:
# # Installed in requirements, if not install here.
# !pip install nltk
# !pip install spacy

In [ ]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image, display

In [ ]:
input_dir = './data/input/'
output_dir = './data/output/psm/'

In [ ]:
from datetime import datetime, timedelta
import datetime as dt
def unixtimestamp_to_readable(unix_timestamp=1284101485):
    
    ts = int(unix_timestamp)

    # if you encounter a "year is out of range" error the timestamp
    # may be in milliseconds, try `ts /= 1000` in that case
    return datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d_%H-%M-%S')


def unixtimestamp_to_datetime(unix_timestamp=1284101485):
    unix_timestamp = int(unix_timestamp)
    dt_object = dt.datetime.fromtimestamp(unix_timestamp)
    return dt_object

def datetime_to_unixtimestamp(datetime_obj):
    timestamp = datetime.timestamp(datetime_obj)
    return int(timestamp)


def clean_text(text):
#     text = text.strip()
    text = re.sub('&amp;', '&', text)
    text = re.sub('&lt;', '<', text)
    text = re.sub('&gt;', '>', text)
    text = re.sub('&#x200B;', '', text)
    text = re.sub('&nbsp;', ' ', text)
    text = text.replace('\n', ' ')
    
    return text


# Load all posts and comments 

In [ ]:
# posts

df = pd.read_csv(input_dir+'first_sw_submission_2021-07-20-23-12-27.csv',index_col=0, lineterminator='\n')
df

In [ ]:

text_all = []
for title, body in zip(df['title'].values, df['selftext'].values):
    title = title.strip()
    if title.endswith('.') or title.endswith('?') or title.endswith('!'):
        title += ' '
    else:
        title += '. '
    text = title+str(body).strip()
    text_all.append(text)

text_all = pd.DataFrame(text_all)
text_all
        

df['text']  = text_all
        
df[['text', 'title', 'selftext']]        

In [ ]:
searchfor = '\n+', '&amp;','&lt;','&gt;','&#x200B;','&nbsp;'
errors = df[df.text.str.contains('|'.join(searchfor))]
print(errors.text.tolist()[:100])


In [ ]:

df['text'] = [clean_text(n) for n in df.text.values]
df



In [ ]:
df.to_csv(input_dir+'first_sw_submission_2021-07-20-23-12-27.csv') 

# Manually run through LIWC2015

In [ ]:
df_original = pd.read_csv(input_dir+'first_sw_submission_2021-07-20-23-12-27.csv', index_col=0, lineterminator='\n')
df_original

In [ ]:
import pandas as pd
input_dir = './data/input/'
# Rename LIWC columns
df_original = pd.read_csv(input_dir+'first_sw_submission_2021-07-20-23-12-27.csv', index_col=0, lineterminator='\n')
df_cols  = df_original.columns.tolist()
df = pd.read_csv(input_dir+'first_sw_submission_liwc2015_2021-07-20-23-12-27.csv', index_col=0)
df.columns = df_cols+df.columns.tolist()[len(df_cols):]
df.to_csv(input_dir+'first_sw_submission_liwc2015_2021-07-20-23-12-27.csv')
df

In [ ]:
input_dir = './data/input/'
# Rename LIWC columns
df_sub = pd.read_csv(input_dir+'first_sw_submission_liwc2015_subsequent_posts_2021-07-20-23-12-27.csv', index_col=0)
liwc = pd.read_csv(input_dir+'first_sw_submission_liwc2015_2021-07-20-23-12-27.csv', index_col=0)


In [ ]:
set(df_sub.id.values)- set(liwc.id.values)

In [ ]:
liwc = liwc[liwc.id != '81ust5']
liwc = liwc[liwc.id != '7xm567']
df_sub = df_sub[df_sub.id != '81ust5']
df_sub = df_sub[df_sub.id != '7xm567']



liwc = liwc.dropna(subset=['id'])
df_sub =df_sub.dropna(subset=['id'])
liwc = liwc.sort_values('author')
df_sub = df_sub.sort_values('author')



In [ ]:
import numpy as np
np.sum(df_sub.id.values == liwc.id.values)

In [ ]:
print(df_sub.shape)
print(liwc.shape)

In [ ]:
common_columns = list(set(liwc.columns).intersection(set(df_sub.columns)))

In [ ]:
df = liwc.merge(df_sub, on=common_columns)
df.to_csv(input_dir+'first_sw_submission_liwc2015_subsequent_posts_2021-07-20-23-12-27.csv')

In [ ]:
comments_original = pd.read_csv(input_dir+'first_sw_comments_2021-07-20-18-52-35.csv', index_col=0)
comments_original['body'] = [clean_text(n) for n in comments_original.body.values]
comments_original.to_csv(input_dir+'first_sw_comments_2021-07-20-18-52-35.csv')

In [ ]:
# Transform through LIWC2015

In [ ]:
comments_original = pd.read_csv(input_dir+'first_sw_comments_2021-07-20-18-52-35.csv', index_col=0)
comments_cols  = comments_original.columns.tolist()
comments = pd.read_csv(input_dir+'first_sw_comments_liwc2015_2021-07-20-18-52-35.csv', index_col=0)
comments.columns = comments_cols+comments.columns.tolist()[len(comments_cols):]
comments = comments.drop('permalink', axis=1)
comments

In [ ]:

comments.to_csv(input_dir+'first_sw_comments_liwc2015_2021-07-20-18-52-35.csv')

In [ ]:
# TEST: passed
# Check that permalinks are correct. I checked. They're correct.
# print(df.title.values[100])
# print(df.full_link.values[100])
# print(df.title.values[20000])
# print(df.full_link.values[20000])

In [ ]:
# # TEST: passed
# # check that for a given post, I have the comments. 
# post_i = 'ar8ydo'
# comments_i = comments[comments.link_id.str.contains(post_i)]
# print('www.reddit.com/'+comments_i.permalink.values[0])

# post_row = df[df.id==post_i]
# content = post_row.title+'\n'+post_row.selftext
# print(content.values[0])
# comments_i

# Custom lexicons

In [ ]:
# write in lowercase, only punctuation are apostrophes like "i'm", mainly oriented to first person singular
# lexicons may assume the document contains complaints and stressors. So it counts 'dad' as domestic_stress_and_violence.
# word stems are included were possible to capture different suffixes

financial_stress = [
'credit card', 'bank', "bad credit","my credit",'loan', 'lend',
'econom',"financ",
'rent', 'mortgage', 'evict','insurance',
"money",
'owe', 'debt', "make ends meet","make ends meat",
'pay off','payment', 'taxes',
'bills','tuition','lose my scholarship',
"afford",
"save enough money", "save money","saving",
'salary', 'wage', 'income', 'job','employ', #unemploy
'foreclosure', 'bankrupt',
'poverty', "poor", 'disadvantaged']

domestic_stress_and_violence = ['divorc', 'marri','marry','widow','violence', 'abuse', 'yelling','screaming',
'cheat','ghost','selfish','self centered',
'boyfriend', 'girlfriend', 'partner', 'husband', 'wife','dad', 'mom', 'brother', 'sister', 'sibling',
"we're fighting", 'single mom', 'single dad', 'single parent', 'hit me',
'slapped me','kick','punch', 'fight','fought', 'aggressi', 'agresi', 'agressi', 'choke','slam', 'push me', 'pushed me',
'forced me to', 'knocked me', 'strike me', 'cut me', 'threat', 'stab', 'control me', 'controlling me',
'knife', 'knives','stab', 'blade', 'gouge', 'dagger','gun', 'pistol', 'revolver', 'semiautomatic', 'semi automatic','rifle', 'shoot', 'firearm', ]

loneliness_isolation = ['alone', 'lonely', 'no one cares about me', 'no one cares', "can't see anyone", 'no one else to talk to',
"can't see my", "i miss my", "i want to see my", "trapped", "i'm in a cage",
"feel ignored", "ignoring me" "ugly", 'rejected', 'avoiding me', " lack of social",'akward',
"no friends", "any friends","make friends", "want someone to","want anyone to", "have no one", "to hang out",
"people in my life","wrong with me",
'am single', 'been single', 'quarantine', 'lockdown', 'isolation', 'self-isolation', 'by myself',
"find someone"]


substance_use = [
'substance','drug',
'addict', 'clean', 'rehab', 'sober', 'relaps','withdraw','12 step',
    'adderal', 'vyvans',
'medicat','lexapro','prozac','zoloft','valium','xanax',
'aa meeting', 'na meeting',
'smoke','vape', 'inject', 'snort','needle',
'drink', 'drunk', 'drank',  'overdos','oding','hung over', 'hangover',
'cigarette', 'tobacco','nicotine','weed','cannabis', 'marihuana','ganja','hashish',
'alcohol','beer',	'vodka', 'whiskey', 'whisky', 'tequila','wine',
'pill','capsule','barbiturate', 'pentobarbital', 'nembutal', 'zann','benzodiazepines', 'alprazolam', 
'chlorodiazepoxide', 'librium', 'diazepam', 
    'lorazepam', 'ativan', 'triazolam', 'halicon'
'eszopiclone', 'lunesta','zaleplon', 'sonata','zolpidem','ambien',
'opioid','cocaine','blow','crack',
'heroine', 'smack','methadone','buprenorphine', 'naltrexone',
'laughing gas', 'nitrite', 'paint thiner', 'nitrous oxide',
'ketamine','special k',
'khat','kratom',
'shrooms','psilocybin', 'mushrooms', 'lsd', 'acid',
'mescaline', 'peyote',
'dxm','loperamide',
'mdma', 'molly', 'ecstasy','dmt',
'ghb', 'xyrem',
'codeine', 'vicodin', 'fentanyl','codone','percocet', 'oxycontin',
'adderall', 'methylphenidate', 'concerta', 'ritalin',
'roofie', 'rohypnol','ruffie',
'testosterone',
'bath salt',
'amphetamine', 'pcp','angel dust', 'stimulant',
'inhalant', 'desoxyn', 'crank', 'chalk', 'gak',  'pookie', 'quartz', 'rocket fuel', 'scooby snax', 'took speed','taking speed', 'take speed' #to dissambiguate with normal speed
'meth']

suicidality_passive = [
#    passive
"don't know how much longer I",
"would be better off dead", "would be better off without me","would be better off if I died", #RISK FALSE POSITIVES
"want to be here anymore","wanna to be here anymore",
"want to die", "wanting to die","wanted to die","wanna die",
"don't want to live", 'not wanting to live',
'want to go to sleep',"wanna go to sleep"
'wish i could go to sleep',
'close my eyes and never open them again',
'not wake up',
"don't wake up",
'never wake up',
'never want to wake up','never wanna wake up',
"don't want to wake up","don't wanna wake up",
'wish it would all end', 'done with living', 'want it to end',
'live anymore', 'living anymore', 'life anymore', 'be dead',
"nothing to live for","nothing left to live for",
'shoot me', 'kill me',"shot me","killed me", 
"don't want to be alive","don't wanna be alive",
'think about death','thinking about death',
"life isn't worth living","life is not worth living",
"i'm ready to go", 'i am ready to go',
'if I live or die', 'I hate my life','if I lived or die', 'I hated my life',
'no point in living',
    "wish i were never born",
    "have no future",   
        'my last day',
        'let me die',
    'let me go',
]

suicidality_general = [
'suicid',
'depress', #depressed, 'depression'
    "shitty life",
    "sick of life",
    "i deserve to die",


    "life is worse",
    "life is over",
    'life is shit',
    'not be here',
    "when I'm not around",

    'crisis center', 'crisis hotline', 'crisis text',
'thing keeping me','only thing that kept me', 'only thing stopping me', #going, alive, from X
'thing that keeps me going','thing that keeps me alive','thing that keeps me from',
"want to do this anymore","wanted to do this anymore","wanna to do this anymore",
"don't know how much longer",

    "i'm worthless",

    "i'm on the ledge",

    
    "say goodbye",'goodbye world',
    'hate myself',
    'hate my life',
    "one choice",
    
    
    'want to end this',
    
    
    # social: burdemness/loneliness
"i'm a burden", "i'm such a burden",
"no one will miss me",
"better off without me",
    "wasting space",
    "nobody wants me here",
# helplessness
    'have no hope', 'having no hope',
    "it's too late",
'hopeless','helpless',
"don't know what to do","don know what to do","dont know what to do","idk what to do", "not knowing what to do",
"don't see the light at the end", "not seeing the light at the end",
"do with my life", "done with my life", #don't know what to do with my life,don't know what I want to do with my life,
'take it anymore',
"point of living",
    "there no point","there's no point",
    
    
]

suicidality_selfharm = ['self harm', 'selfharm',
'self injur','cut my','slit', 'wrist cut','lacerat',
'made cuts on my', 'made a cut on my', 'made a gash', 'gashed my', 'split open my',
'pierced my', 'piercing my', 'pierce my','burn my', 
'cutting',
'hurt myself', 'hurting my','hurt myself',
]

# active
suicidality_active = [
'commit suicide','commiting suicide',
'attempt' #HIGH RISK FOR FALSE POSITIVES
'jump off a bridge','jumping off a bridge',
'throw myself into', #traffic
'drive into something',
'kill myself','killing myself','killed myself','kill yourself',
'hang myself', 'hanging my','hung myself',
'shoot myself', 'shooting myself',
'gun in my mouth',
'overdose','overdosing','oding', 'offing',
'it all ends tonight',
'end my life','ending my life','ended my life',
'want to end it','wanting to end it',"wanna end it","off myself",
    'going to end it','enough to end it all', 'ending it all',
    "dispose of my body",
    'going to die tonight', 'will die tonight', 
    'pull the trigger', 'pulling the trigger',


]


'''
plan: 'suicidal plans', 'suicide plans', 'planning'

attempts: 'i've tried multiple times'

'''

mental_health = [
'in and out of the hospital','hospitalized',
'psychiatr','mentally ill','mental problem','professional help',"i'm not ok",'diagnos','treatment','treated for','get help', 'getting help','therap','neuroti', 'stress','disorder',
'mental health', 'craz','lunatic', 'insane','deranged','demented','unbalanced', 'unstable','suffer','pain',
     'dysregulat', 'affective',   
     'dull','bored',
    'stress','overwhelm',
    "don't know what to do","don know what to do","dont know what to do","idk what to do", "not knowing what to do",
'depress', 'dysthymia', 'MDD','sad','unhappy','exhaust', 'drain', 'ashamed', 'guilt','numb','useless','pathetic self','apath',
'have no motivation','lack motivation',"don't have motivation",'keeps getting worse',
"i'm a failure", "fidgety",
'insomnia', "can't sleep", "not sleeping",
    "help", "need someone to talk", 'talk to someone',
    'angry',
'narcissis','ego','selfish','self centered',
'ptsd', 'trauma', 'flashback', 'trigger','nightmar','post traumatic stress disorder',
'schizophren', 'hallucin', 'delus','hearing voic','heard voic','hear voic', 'paranoid','psycho','deliri',
 'schizo',
'anxiety','anxiou','antsy','anxieti', 'panic','obsess','ocd', 'worried', 'worry', 'nervous', 'phobi',
'restless', 'irrita', 'annoy', 'impulsive', 'rambl',
'autis', 'spectrum', 
'bipolar', 'manic', 'mania', 'lithium','episod',
'restrict', 'purg', 'bing', 'calori', 'recover','vomit','fasted', 'fasting',
'eating disorder', 'anorexi', 'bulimi',
'adhd', 'adderal', 'vyvans',
'medicat','lexapro','prozac','zoloft','valium','xanax',
'cyclothymic',
 'mutism',
 'dysmorphic',
 'hoard',
 'trichotillo',
 'dissociative',
 'depersonaliz',
 'ruminat',
 'personality',
 'antisocial',
 'borderline',
 'histrionic',
 'avoidant',
 'dependent',
 'alogia',
 

]


In [ ]:
import re




    
def count_words(doc,tokens=[], normalize = True):
    # Count number times a specific tokens (word, phrase) appears in the doc.
    text = re.sub("[^\w\d'\s]+",'',doc.lower())    # remove punctuation except apostrophes because we need to search for things like "don't want to live"
    counter = np.sum([text.count(token) for token in tokens])
    if normalize:
        
        word_count = len(re.findall(r'\w+', doc))
        if word_count == 0:
            assert counter == 0
            return 0
        normalized_counter = counter/word_count
        return normalized_counter
    else:
        return counter

    



In [ ]:
import pandas as pd
input_dir = './data/input/'
docs = pd.read_csv(input_dir+'first_sw_submission_liwc2015_subsequent_posts_2021-07-20-23-12-27.csv', index_col = 0)['text'].values
print(len(docs))

In [ ]:

import numpy as np

lexicons = [financial_stress, domestic_stress_and_violence,loneliness_isolation, substance_use,suicidality_general,
    suicidality_selfharm, suicidality_passive, suicidality_active,mental_health]

lexicon_names = ['financial_stress',
 'domestic_stress_and_violence',
 'loneliness_isolation',
 'substance_use',
 'suicidality_general',
 'suicidality_selfharm',
 'suicidality_passive',
 'suicidality_active',
 'mental_health']

In [ ]:
import pandas as pd
d = pd.DataFrame(np.hstack(lexicons)).value_counts()
d[d>1]

In [ ]:
%%time



feature_vectors = []
for i,lexicon in enumerate(lexicons):
    print(i)
    lexicon = [n.lower() for n in lexicon]
    counters = [count_words(doc,tokens=lexicon, normalize=True) for doc in docs]
    feature_vectors.append(counters)

feature_vectors = pd.DataFrame(feature_vectors, index = lexicon_names).T






In [ ]:
feature_vectors['text'] = docs
pd.set_option('max_colwidth', -1)


In [ ]:
feature_vectors.shape[0]

### Check which ngrams were not captured at all

In [ ]:

indexes = feature_vectors[feature_vectors.drop('text', axis=1).sum(axis=1)==0].index
missed_sentences = feature_vectors.loc[indexes].text.tolist()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def count_vectorizer(docs, stop_words='english', ngram_range=(1,1), min_df = 1):

	vectorizer = CountVectorizer(
	    analyzer='word',
	    stop_words = stop_words, #None, #'english'
	    ngram_range=ngram_range,
	    min_df = min_df,
	)         #this is 1 per document, below, I compute total
	X = vectorizer.fit_transform(docs)
	ngram_names = vectorizer.get_feature_names()

	ngram_counts = pd.DataFrame(X.toarray(), columns=ngram_names)
	ngram_counts_sum = pd.DataFrame(ngram_counts.sum()).sort_values(0)[::-1]
	return X, ngram_counts, ngram_counts_sum


In [ ]:

filename_prefix = 'lonely2019'

min_df = 2

top_ngrams = []

for ngram_range in range(2,8):
	if ngram_range  == 1:
		# top ones will just be stopwords
		stop_words = 'english'
	else:
		stop_words = None

	X, ngram_counts, ngram_counts_sum = count_vectorizer(missed_sentences, stop_words=stop_words, ngram_range=(ngram_range,ngram_range), min_df = min_df )
	top_ngrams_i = ngram_counts_sum.reset_index()
	top_ngrams_i.columns = [f'ngram_range_{ngram_range}', f'norm_freq_{ngram_range}']
# 	top_ngrams_i[f'norm_freq_{ngram_range}'] = top_ngrams_i[f'norm_freq_{ngram_range}']/top_ngrams_i[f'norm_freq_{ngram_range}'].sum()
	top_ngrams_i = top_ngrams_i.reset_index(drop=True)
	top_ngrams.append(top_ngrams_i)


In [ ]:
top_ngrams_df = pd.concat(top_ngrams, ignore_index=False, axis=1)
top_ngrams_df.to_csv(input_dir+'missed_ngrams.csv')

# after adding missing ngrams, concat new features 

In [ ]:
print(len(docs))
df = pd.read_csv(input_dir+'first_sw_submission_liwc2015_subsequent_posts_2021-07-20-23-12-27.csv', index_col = 0)
print(df.shape[0])

In [ ]:
df.author.value_counts().sort_values()

In [ ]:
feature_vectors

In [ ]:
df = df.merge(feature_vectors,on='text')


In [ ]:
df.shape

In [ ]:
# Any duplicates?
pd.DataFrame(df.text.value_counts()).iloc[:15]

In [ ]:
df = df.drop_duplicates(subset = 'text',)
print(df.shape)
df.to_csv(input_dir+'first_sw_submission_liwc2015_subsequent_posts_lexicons_2021-07-20-23-12-27.csv')

In [ ]:
df.iloc[::1000, :].to_csv(input_dir+'screen_feature_extraction_random_subset.csv')

In [ ]:
feature_vectors.drop(['text'], axis=1).sum().sort_values()[::-1].round(1)

# extract features for second post

In [ ]:
input_dir = './data/input/'
subs = pd.read_csv(input_dir+'all_submissions_2021-07-20-23-11-56.csv', index_col = 0)


In [ ]:
subs = subs.sort_values(['author', 'created_utc'])
subs = subs[subs.subreddit == 'SuicideWatch']
first_sw = []
second_sw = []
for i, author in enumerate(subs.author.unique()):
    if i%1000==0:
        print(i)
    subs_i = subs[subs.author ==author]
    first_sw.append(
        pd.DataFrame(subs_i.iloc[0,:]) #append first SW post
    )
    if subs_i.shape[0]>1:

        second_sw.append(
            pd.DataFrame(subs_i.iloc[1,:]) #append second SW post
        )
        
        




In [ ]:
first_sw_df = pd.concat(first_sw,axis=1).T
second_sw_df = pd.concat(second_sw,axis=1).T
second_sw_df

In [ ]:
# # TEST PASSED: the original dataset contains all the first_sw_df posts: 
# original = pd.read_csv(input_dir+'first_sw_submission_liwc2015_subsequent_posts_lexicons_2021-07-20-23-12-27.csv', index_col = 0)
# print(first_sw_df.shape, original.shape)
# print(len(set(original.id.values)-set(first_sw_df.id.values)))
# print(len(set(first_sw_df.id.values)-set(original.id.values)))

In [ ]:
second_sw_df = second_sw_df.reset_index(drop=True)
text_all = []
for title, body in zip(second_sw_df['title'].values, second_sw_df['selftext'].values):
    title = title.strip()
    if title.endswith('.') or title.endswith('?') or title.endswith('!'):
        title += ' '
    else:
        title += '. '
    text = title+str(body).strip()
    text_all.append(text)

text_all = pd.DataFrame(text_all)
text_all
        

second_sw_df['text']  = text_all
        
second_sw_df[['text', 'title', 'selftext']]     



In [ ]:
%%time
docs = second_sw_df.text.values


feature_vectors = []
for i,lexicon in enumerate(lexicons):
    print(i)
    lexicon = [n.lower() for n in lexicon]
    counters = [count_words(doc,tokens=lexicon, normalize=True) for doc in docs]
    feature_vectors.append(counters)

feature_vectors = pd.DataFrame(feature_vectors, index = lexicon_names).T






feature_vectors['text'] = docs
pd.set_option('max_colwidth', -1)


In [ ]:
lexicon_names

In [ ]:
df.describe()

In [ ]:
df = second_sw_df.merge(feature_vectors,on='text')
df


In [ ]:
df.to_csv(input_dir+'second_sw_submission_lexicons_2021-07-20-23-12-27.csv')

# Descriptive statistics

In [ ]:
df = pd.read_csv(input_dir+'first_sw_submission_liwc2015_subsequent_posts_lexicons_2021-07-20-23-12-27.csv', index_col = 0)
df.shape

In [ ]:

#TODO: maybe post to github



# Reddit
print('SW posts: ',df[df.subreddit =='SuicideWatch'].shape[0])
print('Non SW posts: ',df[df.subreddit !='SuicideWatch'].shape[0])
print('SW post rate: ',df[df.subreddit =='SuicideWatch'].shape[0]/df.shape[0])
print('unique posters: ',len(df.author.unique()) )


In [ ]:
comments.author.value_counts().describe().astype(int)

In [ ]:
# Build dataset of first SW posts 

In [ ]:
'''
SITBI poster: person who posts SITBI-present
commenter: someone who answers to a SITBI post

1. Find first suicidal post and make sure first post is in 2018 if not remove.
2. Then add a col "relapse" with 1 if they post SITB again and 2 if they do not. 
3. Discard prior posts: maybe keep them as covariates. Should we keep prior comments?
4. Feature extraction: ngrams, features from comments, maybe features from profile, amount of commenters


alternative covariates: instead being just the ngram, it could be a feature vector for each word. have they interacted before?
simpler treatment: amount of responses
alternative treatment: user, what type of user improves outcome
alternative outcome: do they *not* post. This could be bad. 

'''

In [ ]:
all_subs = pd.read_csv(input_dir+'all_submissions_2021-07-20-23-11-56.csv', index_col = 0)
all_subs

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'


In [ ]:
all_subs.created_utc

In [ ]:
print(all_subs.shape)
all_subs = all_subs[~all_subs.created_utc.isnull()]
print(all_subs.shape)

In [ ]:
all_subs = all_subs[all_subs.created_utc.apply(pd.to_numeric, errors='coerce').notna()].dropna(subset=['created_utc'])
all_subs.created_utc = all_subs.created_utc.astype(int)
print(all_subs.shape)


In [ ]:
days = [7,14,21,30,60,90,180,365]
run = True
re_save_df = True


if run:
    first_sw_post = []
    posts_after_2018_dropped = []
    no_sw_posts = 0
    label = 'SuicideWatch' #maybe I want to add r/Depression or other mental health subreddits
    total = all_subs.author.unique().shape[0]
    print(total)

    for i, author in enumerate(all_subs.author.unique()):
        if i%50==0:
            print(i)
        subsequent_sw=0
        # posts by author     
        df_author = all_subs[all_subs.author==author]
        df_author = df_author.sort_values('created_utc') #make sure its ordered by date (oldest first) because we want first suicidal post
        suicide_posts = df_author[df_author['subreddit']==label]
        if suicide_posts.shape[0]==0:
            # no suicide_posts, I selected these authors from Reddit Mental Health Dataset SuicideWatch from 2018 and 2019, but their suicide posts were not found through the API?
            no_sw_posts += 1
            continue

        # keep first post (they're sorted chronologically)    
        suicide_posts_first = suicide_posts.drop_duplicates(subset = 'author',keep='first') 
        utc_first_sw_post = suicide_posts_first.created_utc.values[0]
        id_first_sw_post = suicide_posts_first.id.values[0]

        # TEST: PASSED
        #double check if dates are in right order
    #     assert df_author.sort_values('created_utc').equals(df_author) 
    #     print('\n'+df_author.author.values[0])
    #     print(date_first_sw_post.values[0])

        #obtain all subsequent and prior posts #TODO: Double check this, because it says 4
        df_author_subsequent = df_author[df_author.created_utc>utc_first_sw_post]
        df_author_prior = df_author[df_author.created_utc<utc_first_sw_post]


        if df_author.shape[0]>1:
            # all subsequent posts
            subreddits_counts = df_author_subsequent.subreddit.value_counts()
            if label in subreddits_counts.index: #ToDo maybe include label or 'depression'
                subsequent_sw = subreddits_counts[label]
            else:
                subsequent_sw = 0
        else:
            assert df_author.shape[0] == 1# there was only that first sw post, maybe some posts before, but none after.         
            subsequent_sw = 0# did not post again after SW post


        last_post_date = df_author.iloc[-1].created_utc
        suicide_posts_first = suicide_posts_first.reset_index(drop=True)
        suicide_posts_first.loc[0,'subsequent_sw']=subsequent_sw
        suicide_posts_first.loc[0,'last_post_date']=last_post_date
        suicide_posts_first.loc[0,'prior_posts']=df_author_prior.shape[0]
        suicide_posts_first.loc[0,'subsequent_posts']=df_author_subsequent.shape[0]

        for n in days:
            cutoff_day_datetime = unixtimestamp_to_datetime(utc_first_sw_post)+timedelta(days=n)
            cutoff_day_unix = int(datetime_to_unixtimestamp(cutoff_day_datetime))
            df_author_subsequent_n = df_author[(df_author.created_utc>utc_first_sw_post)&(df_author.created_utc<=cutoff_day_unix)]
            subreddits_counts_n = df_author_subsequent_n.subreddit.value_counts()
            if label in subreddits_counts_n.index: #ToDo maybe include label or 'depression'
                subsequent_sw_n = subreddits_counts_n[label]

            else:
                subsequent_sw_n= 0
            suicide_posts_first.loc[0,f'subsequent_sw_{n}days']=subsequent_sw_n
            suicide_posts_first.loc[0,f'subsequent_posts_{n}days']=df_author_subsequent_n.shape[0]

        first_sw_post.append(suicide_posts_first)

    first_sw_post = pd.concat(first_sw_post)
    first_sw_post = first_sw_post.reset_index(drop=True)
    first_sw_post['created_utc_readable'] = [unixtimestamp_to_readable(j) for j in first_sw_post.created_utc.values]

    
    if re_save_df:

        timestamp ='{:%Y-%m-%d-%H-%M-%S}'.format(datetime.now())


        first_sw_post.to_csv(input_dir+f'first_sw_submission_liwc2015_subsequent_posts_2021-07-20-23-12-27_{timestamp}.csv', )




In [ ]:
first_sw_post

In [ ]:
# Resave all_subs without NaNs in created_utc. AND SEE WHAT OTHER NANS THERE OR PER COL
all_subs.isna()2.sum()

In [ ]:
# Resave all_subs without NaNs in created_utc. AND SEE WHAT OTHER NANS THERE OR PER COL
all_subs.isna().sum()

In [ ]:

all_subs.to_csv(input_dir+'all_submissions_2021-07-20-23-11-56.csv')


# More descriptive stats

In [ ]:
input_dir = './data/input/'
output_dir = './data/output/'

In [ ]:


first_sw_post = pd.read_csv(input_dir+'first_sw_submission_liwc2015_subsequent_posts_lexicons_2021-07-20-23-12-27.csv', index_col = 0)
comments = pd.read_csv(input_dir+'first_sw_comments_liwc2015_2021-07-20-18-52-35.csv', index_col = 0)


In [ ]:
first_sw_posts.

In [ ]:

# print('users without sw: ', no_sw_posts) #look into this, i selected them from Reddit Mental Health dataset
print(f'Result: {first_sw_post.shape[0]} unique users and { len(first_sw_post.author.unique())} unique posts') 
# print('users with subsequent sw: ', first_sw_post.shape[0])



In [ ]:
stats_comments = pd.DataFrame(comments.author.value_counts().iloc[1:].describe()).T
stats_comments

In [ ]:

columns_stats = ['prior_posts','subsequent_posts','subsequent_sw', 'num_comments']
stats = first_sw_post[columns_stats].describe().round(1).T
stats

In [ ]:
stats_comments.values

In [ ]:
stats.loc[len(stats)] = stats_comments.values[0]
stats = stats[['min', '25%', '50%', '75%', 'max']].astype(int)
stats.index = ['Prior posts', 'All subsequent posts', 'Subsequent SW posts', 'Comments per post', 'Comments per commenter']
stats

In [ ]:
print(stats.to_latex())

In [ ]:
sw_counts = first_sw_post.prior_posts.value_counts().reset_index()
sw_counts


In [ ]:
for i in range(2):
    count = sw_counts[sw_counts['index']==i].prior_posts.values
    print((count/df.shape[0]).round(3)*100, f'(N={count})')



In [ ]:
subsequent_count = first_sw_post.subsequent_posts.value_counts().reset_index()
subsequent_count

In [ ]:
4207/df.shape[0]

### Subsequent days

In [ ]:
days = [7,14,21,30,60,90,180,365]
subsequent_df = []
for n in days:
    subsequent = pd.DataFrame(first_sw_post[f'subsequent_sw_{n}days'].value_counts())
    subsequent_df.append(subsequent)

subsequent_df = pd.concat(subsequent_df, axis=1)
subsequent_df = subsequent_df.fillna(0)
subsequent_df = subsequent_df.reset_index()




subsequent_df = subsequent_df.rename(columns = {'index':'posts'})
subsequent_df

In [ ]:
subsequent_df

In [ ]:
subsequent_df[subsequent_df['posts']>7].sum()[1:]

In [ ]:
print(subsequent_df[subsequent_df['posts']0].sum()[1:])
(subsequent_df[subsequent_df['posts']!=0].sum()[1:]/df.shape[0]).round(3)*100

In [ ]:
print(subsequent_df[subsequent_df['posts']!=0].sum()[1:])
(subsequent_df[subsequent_df['posts']!=0].sum()[1:]/df.shape[0]).round(3)*100

In [ ]:
size=9
rotation=90
round_int = 3


plt.figure(
    figsize=(8,4), 
    dpi=300)

# 1 or more
x = days
# y = subsequent_df[subsequent_df['posts']!=0].sum()[1:]
y = (subsequent_df[subsequent_df['posts']!=0].sum()[1:]/df.shape[0]).round(round_int)

plt.scatter(days,y)
plt.plot(days,y, label = '1 or more')

# 1 to 7
for n in range(1,7):
#     y = subsequent_df[subsequent_df['posts']==n].values[0][1:]
    y = (subsequent_df[subsequent_df['posts']==n].values[0][1:]/df.shape[0]).round(round_int)
    
    plt.scatter(days,y)
    plt.plot(days,y, label = n)
    
# y = subsequent_df[subsequent_df['posts']>=7].sum()[1:]
# plt.scatter(days,y)
# plt.plot(days,y, label = '7 or more')



l = plt.legend(bbox_to_anchor=(1.05, 1), title = 'Subsequent\nr/SuicideWatch\nposts', framealpha=1)
plt.setp(l.get_title(), multialignment='center')



plt.xlabel('Days')
plt.ylabel('Proportion of users')
plt.xticks(ticks=days, labels=days, rotation=rotation, size=size)
plt.tight_layout()
plt.savefig(output_dir+'figures/subsequent_sw_posts.png', dpi=300, bbox_inches='tight')
plt.show()    



In [ ]:

plt.figure(
#     figsize=(3,2), 
    dpi=300)

for n in [0]:
    y = subsequent_df[subsequent_df['posts']==n].values[0][1:]
    plt.scatter(days,y)
    plt.plot(days,y, label = n)

plt.legend(
#     bbox_to_anchor=(1.05, 1), 
    title = 'Subsequent\nr/SuicideWatch\nposts')
plt.xlabel('Days')
plt.ylabel('Users')
plt.xticks(ticks=days, labels=days, rotation=rotation, size=size)
plt.ylim(0,21100)
plt.show()    


### Prior posts

In [ ]:
prior_posts = first_sw_post.prior_posts
prior_posts.head()

In [ ]:
prior = pd.DataFrame(first_sw_post.prior_posts.value_counts())

prior = prior.sort_index().reset_index()
prior.columns = ['Prior posts', 'Users']
prior = prior.astype(int)
prior['Users %'] = (prior['Users']/prior['Users'].sum()).round(3)

prior.iloc[:21]

In [ ]:
prior_posts.describe().astype(int)

In [ ]:
np.max(prior_posts)

In [ ]:
import seaborn as sns
sns.histplot(prior_posts, bins = int(np.max(prior_posts)))
plt.xlim(-1,100)



In [ ]:
pd.DataFrame(prior_posts.describe().astype(int))


In [ ]:
subsequent = pd.DataFrame(first_sw_post.subsequent_posts.value_counts())
subsequent_posts = pd.DataFrame(first_sw_post.subsequent_posts)

import seaborn as sns
sns.histplot(subsequent_posts, bins = int(np.max(subsequent_posts))+1, legend=False)

plt.xlim(-1,100)